In [449]:
import pandas as pd
from sklearn.svm import SVR
import numpy as np

def mape(y_pred, y_true):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [474]:
dataset = pd.read_csv("../data/preprocessed_input_interpolate_20min_phase1and2_train.csv")
del dataset['date']

# Only select columns which are useful
selected_col = ["hour", "precipitation", "dayofweek", "is_holiday"]

# Target predict column
predict_col = "(3, 0, 'tot')"

# Time shift backward for predict_col
dataset_shift_1 = dataset.shift(periods=1)[predict_col].fillna(method="bfill")
dataset_shift_2 = dataset.shift(periods=2)[predict_col].fillna(method="bfill")
dataset_shift_6 = dataset.shift(periods=6)[predict_col].fillna(method="bfill")

# Add time shifted predict_col to dataset
#dataset = dataset.assign(predict_shift_1=pd.Series(dataset_shift_1).values)
#dataset = dataset.assign(predict_shift_2=pd.Series(dataset_shift_2).values)
dataset = dataset.assign(predict_shift_6=pd.Series(dataset_shift_6).values)
#selected_col.append(predict_col)
#selected_col.append("predict_shift_1")
#selected_col.append("predict_shift_2")
#selected_col.append("predict_shift_6")

training_set = dataset[:-24*3*7]
predict_set = dataset[-24*3*7:][lambda df : ((df.hour >= 8) & (df.hour < 10)) | ((df.hour >= 17) & (df.hour < 19))]
#training_set[selected_col]
#predict_set[selected_col]

# Prepare potential features to add to the feature set
if predict_col == "(1, 0, 'tot')":
    unused_features = ["(1, 0, 'cargocar')", "(1, 0, 'etc')", "(1, 0, 'motorcycle')", "(1, 0, 'privatecar')", "(1, 0, 'tot')", "(1, 0, 'unknowncar')"]
elif predict_col == "(1, 1, 'tot')":
    unused_features = ["(1, 1, 'cargocar')", "(1, 1, 'etc')", "(1, 1, 'motorcycle')", "(1, 1, 'privatecar')", "(1, 1, 'tot')", "(1, 1, 'unknowncar')"]
elif predict_col == "(2, 0, 'tot')":
    unused_features = ["(2, 0, 'cargocar')", "(2, 0, 'etc')", "(2, 0, 'motorcycle')", "(2, 0, 'privatecar')", "(2, 0, 'tot')", "(2, 0, 'unknowncar')"]
elif predict_col == "(3, 0, 'tot')":
    unused_features = ["(3, 0, 'cargocar')", "(3, 0, 'etc')", "(3, 0, 'motorcycle')", "(3, 0, 'privatecar')", "(3, 0, 'tot')", "(3, 0, 'unknowncar')"]
elif predict_col == "(3, 1, 'tot')":
    unused_features = ["(3, 1, 'cargocar')", "(3, 1, 'etc')", "(3, 1, 'motorcycle')", "(3, 1, 'privatecar')", "(3, 1, 'tot')", "(3, 1, 'unknowncar')"]
unused_features.extend(selected_col)
potential_features = list(training_set.columns)
potential_features = [x for x in potential_features if x not in unused_features]

In [475]:
clf = SVR(C=1.0, epsilon=0.1)
clf.fit(training_set[selected_col], training_set[predict_col])
base_score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col])
print(base_score)

15.032769179525243


In [476]:
next_feature = None

while True:
    for col in potential_features:
        selected_col.append(col)
        clf.fit(training_set[selected_col], training_set[predict_col])
        score = mape(clf.predict(predict_set[selected_col]), predict_set[predict_col])
        if base_score > score:
            base_score = score
            next_feature = col
        selected_col.pop()
    if next_feature is not None:
        selected_col.append(next_feature)
        print(next_feature, base_score)
        potential_features.remove(next_feature)
        next_feature = None
    else:
        break
        
print(selected_col)

wind_speed 14.795481308068812
(1, 0, 'cargocar') 14.692872713188759
['hour', 'precipitation', 'dayofweek', 'is_holiday', 'wind_speed', "(1, 0, 'cargocar')"]
